In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import chain


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3, max_tokens=1000)


# general information
url_list = [
    "https://apnews.com/article/sri-lanka-presidential-election-explained-wickremesinghe-e203abecd36d04fde21e927b9be8ba71",
    "https://www.electionguide.org/elections/id/4364/",
    "https://thediplomat.com/2024/08/sri-lankas-anura-dissanayake-a-strong-contender-for-president/",
    "https://www.isas.nus.edu.sg/papers/sri-lanka-election-2024-key-political-candidates-and-campaigns/",
    "https://www.thehindu.com/news/international/in-southern-sri-lanka-a-chant-for-change-this-poll-season/article68628670.ece",
    "https://groundviews.org/2024/09/04/the-hard-truth-supporting-ranil-now-could-trigger-the-next-crisis/",
    "https://groundviews.org/2024/08/29/the-jvp-and-ethnic-relations-walking-a-tightrope-to-2024-part-1/",
    "https://groundviews.org/2024/09/10/changes-in-support-for-the-leading-presidential-candidates/",
    "https://groundviews.org/2024/09/13/voters-want-more-government-spending-on-health-and-education/",
    "https://groundviews.org/2024/07/18/a-k-dissanayake-continues-to-lead-in-favourability-ratings/",
    "https://groundviews.org/2024/07/22/the-three-way-contest-to-win-the-presidency/",
    "https://www.vifindia.org/article/2024/september/05/Sri-Lanka-at-the-Crossroads-The-Presidential-Election-2024",
    "https://www.dailymirror.lk/breaking-news/Anura-rebuffs-Ranils-proposal-to-invite-IMF-for-debate/108-291616"
]
general_loader = WebBaseLoader(url_list)
general_data = general_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
general_docs = text_splitter.split_documents(general_data)
vectorstore_general = Chroma.from_documents(general_docs, embeddings, collection_name="general")
retriever_general = vectorstore_general.as_retriever(search_kwargs={"k": 10})


# sajith's manifesto
sajith_loader = PyPDFLoader("Sajith_manifesto_english.pdf")
sajith_data = sajith_loader.load()
sajith_docs = text_splitter.split_documents(sajith_data)
vectorstore_sajith = Chroma.from_documents(sajith_docs, embeddings, collection_name="sajith_premadasa")
retriever_sajith = vectorstore_sajith.as_retriever(search_kwargs={"k": 10})

# akd's manifesto
akd_loader = PyPDFLoader("AKD Manifesto.pdf")
akd_data = akd_loader.load()
akd_docs = text_splitter.split_documents(akd_data)
vectorstore_akd = Chroma.from_documents(akd_docs, embeddings, collection_name="anura_kumara_dissanayake")
retriever_akd = vectorstore_akd.as_retriever(search_kwargs={"k": 10})

# ranil's manifesto
ranil_loader = PyPDFLoader("ranil_manifesto_compressed-output.pdf")
ranil_data = ranil_loader.load()
ranil_docs = text_splitter.split_documents(ranil_data)
vectorstore_ranil = Chroma.from_documents(ranil_docs, embeddings, collection_name="ranil_wickramasinghe")
retriever_ranil = vectorstore_ranil.as_retriever(search_kwargs={"k": 10})

retrievers = {
    "sajith_premadasa": retriever_sajith,
    "anura_kumara_dissanayake": retriever_akd,
    "ranil_wickramasinghe": retriever_ranil,
}

class SearchAndCompare(BaseModel):
    """Search for information about a person or compare informations about persons."""

    queryType: str = Field(
        ...,
        description="Query type. Should be `search` or `compare` or `general`. if there's only one person name it's search, if there are many person's name it's compare, or it can be a general question which does not require any specific person",)

    query: str = Field(
        ...,
        description="Query to look up or query to compare",
    )

    candidates: int = Field(
        ...,
        description="Number of persons to search or compare. can be 0 for general questions",
    )

    person1: str = Field(
        ...,
        description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
    )
    person2: str = Field(
        ...,
        description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
    )
    person3: str = Field(
        ...,
        description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
    )

system_query = """You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons."""
prompt_query = ChatPromptTemplate.from_messages(
    [
        ("system", system_query),
        ("human", "{question}"),
    ]
)

structured_llm_query = llm.with_structured_output(SearchAndCompare)
query_analyzer = {"question": RunnablePassthrough()} | prompt_query | structured_llm_query

@chain
def qa_chain(question):
    response = query_analyzer.invoke(question)
    if response.queryType == "search" or response.queryType == "compare":
        if response.queryType == "search":
            retriever = retrievers[response.person1]
            retrieved_docs = retriever.invoke(response.query)

            prompt = (
            "system :"
            "You are an assistant for question-answering tasks. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know."
            "\n\n"
            "{context}"
            "\n\n"

            "human :"
            "{question}"
            ).format(context=retrieved_docs, question=question)

            result = llm.invoke(prompt)

            return result
    
        elif response.queryType == "compare":
            retriever1 = retrievers[response.person1]
            retrieved_docs1 = retriever1.invoke(response.query)

            if response.person2 != 'null':
                retriever2 = retrievers[response.person2]
                retrieved_docs2 = retriever2.invoke(response.query)
            else:
                retrieved_docs2 = ''

            if response.person3 != 'null':
                retriever3 = retrievers[response.person3]
                retrieved_docs3 = retriever3.invoke(response.query)
            else:
                retrieved_docs3 = ''

            prompt = (
            "system :"
            "You are an assistant for comparing manifestos. "
            "Use the following pieces of retrieved context from different manifestos to answer "
            "the question. If you don't know the answer, say that you "
            "don't know."
            "\n\n"
            "{context1}"
            "\n\n"
            "{context2}"
            "\n\n"
            "{context3}"
            "\n\n"

            "human :"
            "{question}"
            ).format(context1=retrieved_docs1, context2=retrieved_docs2, context3=retrieved_docs3, question=question)

            result = llm.invoke(prompt)

            return result
    else:
        retriever = retriever_general
        retrieved_docs = retriever.invoke(response.query)

        prompt = (
            "system :"
            "You are an assistant for question-answering tasks related to srilankan election."
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know."
            "or if the question is not much related to srilankan election say that this question is not related to srilankan election ass a election chatbot i can't provide you with answer this."
            "\n\n"
            "{context}"
            "\n\n"

            "human :"
            "{question}"
            ).format(context=retrieved_docs, question=question)

        result = llm.invoke(prompt)
        return result

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
question = "who is sajeev"
print(qa_chain.invoke(question).content)

This question is not related to the Sri Lankan election. As an election chatbot, I can't provide you with an answer to this. 



In [7]:
question = "who is the father of physics"
print(qa_chain.invoke(question).content)

This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. 



In [8]:
question = "when is sl election"
print(qa_chain.invoke(question).content)

The Sri Lankan presidential election is scheduled for **September 21, 2024**. 



In [9]:
question = "updates on sl election"
print(qa_chain.invoke(question).content)

The Sri Lankan Presidential election is scheduled for 2024.  Here are some updates:

* **Support for Ranil Wickremesinghe has surged:**  Recent polls show a significant increase in support for the incumbent President, Ranil Wickremesinghe.
* **Close race between Premadasa and Dissanayake:** Sajith Premadasa (SJB) and Anura Kumara Dissanayake (NPP/JVP) are currently leading the polls, neck and neck.
* **SLPP support remains low:** Support for a generic SLPP candidate remains unchanged at 4%, with Namal Rajapaksa only declaring his candidacy in August.
* **Party crossovers and changes in political loyalties:**  There has been an unprecedented trend of party crossovers and changes in political loyalties, with many SLPP MPs switching sides to support Wickremesinghe, Premadasa, or Dilith Jayaweera's Mawbima Janatha Party (MJP).
* **Manifestos released:**  Namal Rajapaksa has released his manifesto, "Namal Dekma," while Sajith Premadasa is expected to release his manifesto, "Win For All," so

In [3]:
question = "who is ranil"
print(qa_chain.invoke(question).content)

Ranil is a presidential candidate in the Sri Lankan election. He is being criticized for his age and some of his statements. Some people believe he is not fit to be president, while others support him and believe he has a plan for the future of the country. 



In [4]:
question = "sajith's manifestos about education"
print(qa_chain.invoke(question).content)

Sajith's manifesto focuses on revamping the education sector to build internationally competitive human capital.  Here are some key points:

* **Digital Learning:** Expanding the "Sakwala" program to provide digital learning platforms in schools, technical and vocational institutions, and universities.
* **Early Childhood Education (ECE):** Establishing a regulatory authority for ECE to develop responsible citizens.
* **Teacher Support:** Providing free public transportation to all teachers across Sri Lanka and addressing salary, pension, and service issues.
* **Smart Schools:** Transforming every school into a smart school by providing 100% electricity, water facilities, and physical resources.
* **STEEAM Education:** Prioritizing Science, Technology, English, Engineering, Arts, and Mathematics in all institutions of education.
* **University Reform:** Restructuring the University Grants Commission to address professional issues and strengthening the independence of the University.
* 

In [6]:
question = "compare education policies of sajith and anura"
print(qa_chain.invoke(question).content)

Both Sajith and Anura prioritize education in their manifestos, but their approaches differ in focus and implementation. 

**Sajith's Manifesto:**

* **Focus on accessibility and quality:** Sajith emphasizes making education accessible to all by providing free transportation for teachers, ensuring access to digital learning platforms, and upgrading infrastructure. He also aims to improve quality by reforming the Grade 5 Scholarship Exam, strengthening early childhood education, and promoting English language proficiency.
* **Emphasis on STEM and digital skills:** Sajith proposes a "STEEAM" education system to prioritize Science, Technology, Engineering, Arts, and Mathematics, and plans to expedite teacher training in these areas. He also emphasizes digital learning and the use of technology in education.
* **Specific initiatives:** Sajith outlines specific initiatives like the "Sakwala" program for digital learning, the "Bim Saviya" program for human capital development, and a monthly 